In [105]:
import pandas as pd

In [106]:
"""
Loads in raw score file
Expecting the schema 
Name	Date	GF	GA	Team

Calcuates result, margies and goal difference
Does some prep work for Form and Win

Obviously names need to be consistant
"""

raw_data = pd.read_csv("../data/results_v2.csv")
raw_data["GD"] = raw_data["GF"] - raw_data["GA"]
raw_data.loc[(raw_data['GD'] == 0), 'GamePoints'] = 0.5
raw_data.loc[(raw_data['GD'] < 0), 'GamePoints'] = -1
raw_data.loc[(raw_data['GD'] > 0), 'GamePoints'] = 1
raw_data.loc[(raw_data['GD'] > 6), 'GamePoints'] = 1.5
raw_data.loc[(raw_data['GD'] < -6), 'GamePoints'] = -1.5
raw_data.loc[(raw_data['GamePoints'] >= 1), 'W'] = 1
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'D'] = 1
raw_data.loc[(raw_data['GamePoints'] <= -1), 'L'] = 1
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'MW'] = 1
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'ML'] = 1

raw_data.loc[(raw_data['GamePoints'] >= 1), 'Result'] = '🔼'
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'Result'] = '➖'
raw_data.loc[(raw_data['GamePoints'] <= -1), 'Result'] = '🔽'
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'Result'] = '⏫'
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'Result'] = '⏬'

raw_data = raw_data[raw_data["Name"].str.contains("\*")==False] # Removes Ringers

raw_data = raw_data.fillna(0)
pd.set_option('display.float_format', '{:.1f}'.format)
raw_data = raw_data.astype({'W':'int','D':'int','L':'int','MW':'int','ML':'int'})

raw_data['Date'] = pd.to_datetime(raw_data['Date'],dayfirst = True)
gameweek_int = raw_data.nunique().Date
gameweek = "Gameweek " + str(gameweek_int)

pd.set_option('display.max_rows', 500)

#display(raw_data)

In [107]:
"""
Creates the Form Column
"""
def form_column(raw_data):
    raw_data['game_rank'] = raw_data.sort_values("Date").groupby("Name").Date.rank(ascending=False)
    raw_data_form = raw_data.loc[raw_data['game_rank'] <= 5]
    form = raw_data_form.groupby(['Name'], as_index=False).agg({'Result': ' '.join})
    return form

form = form_column(raw_data)

#display(form)

In [108]:
"""
Creates Matches Played (MP), Partipation (Par%), Pts and Win% (W%)

It also aggreates all the raw data to create an up to date table
"""
def get_totals(raw_data):
    played = raw_data.groupby('Name').count()
    played = pd.DataFrame(played["Date"]).rename(columns={"Date":"MP"})
    data = raw_data.groupby('Name').sum(numeric_only=True).sort_values(by=['GamePoints'],ascending=False).join(played).rename(columns={"GamePoints":"Pts"})

    data["Par%"] = data["MP"].div(gameweek_int)*100
    data["W%"] = data["W"]/data["MP"]*100
    data = data.astype({'Par%':'int','W%':'int'})
    return data

totals = get_totals(raw_data)



#display(data)


In [109]:
"""
Join tables 
Drops unwanted columns 
Puts columns in the correct order
"""
table = totals.merge(form,on='Name').rename(columns={"Result":"Form"})
table.index = table.Name
table = table[['MP', 'W', 'D', 'L','MW', 'ML', 'GF', 'GA','GD','Par%','W%','Pts','Form']]
table.index.names = [gameweek]
table

,MP,W,D,L,MW,ML,GF,GA,GD,Par%,W%,Pts,Form
Gameweek 37,,,,,,,,,,,,,
Calz,23,14,1,8,6,1,350,286,64,62,60,9.0,🔽 🔼 🔽 🔼 ⏫
Gazprom,24,15,0,9,3,3,343,320,23,64,62,6.0,🔽 🔼 🔼 🔼 ⏫
Keysey,16,9,1,6,2,1,233,209,24,43,56,4.0,⏫ 🔼 🔼 🔼 🔼
Virlo,14,8,1,5,3,2,211,192,19,37,57,4.0,🔽 🔽 🔼 🔼 ⏫
Reaper,31,18,0,13,3,6,425,437,-12,83,58,3.5,🔼 🔽 🔼 🔽 ⏬
Schurrle,23,13,0,10,4,3,336,325,11,62,56,3.5,⏫ 🔽 🔽 🔼 ⏬
Peggy,6,3,0,3,1,0,90,79,11,16,50,0.5,🔽 🔼 ⏫ 🔽 🔼
Kim,32,15,1,16,4,4,445,456,-11,86,46,-0.5,🔼 🔼 🔼 🔼 ⏫
Zola,22,10,1,11,3,4,295,325,-30,59,45,-1.0,⏬ 🔼 🔽 🔽 ⏬


In [110]:
"""
TO DO

Add Graphics
Tracking player pairings
Tracking week by week table position

"""


'\nTO DO\n\nAdd Graphics\nTracking player pairings\nTracking week by week table position\n\n'

In [136]:
#display(raw_data)

In [112]:
teams = raw_data.groupby(['Date','Team','Result'])['Name'].apply(','.join).reset_index()
teams[['Player1','Player2','Player3','Player4','Player5']] = teams['Name'].str.split(',',expand=True)
teams = teams.drop('Name',axis=1)
print(teams)

         Date     Team Result   Player1   Player2   Player3   Player4  \
0  2023-03-02  BLANCOS      🔼     Gilet    Reaper  Schurrle    Keysey   
1  2023-03-02    DARKS      🔽      Calz    Helmet       Kim      Mate   
2  2023-03-09  BLANCOS      ⏫      Calz       Kim     Virlo   Gazprom   
3  2023-03-09    DARKS      ⏬     Gilet    Helmet      Mate      Noel   
4  2023-03-16  BLANCOS      ⏬       Kim      Mate      Noel    Reaper   
5  2023-03-16    DARKS      ⏫     Gilet  Schurrle     Virlo   Gazprom   
6  2023-03-23  BLANCOS      🔼      Calz    Reaper   Gazprom      None   
7  2023-03-23    DARKS      🔽      Mate      Noel       Kim     Snoop   
8  2023-03-30  BLANCOS      🔼      Calz      Mate    Reaper      None   
9  2023-03-30    DARKS      🔽       Kim      Noel     Peggy    Helmet   
10 2023-04-06  BLANCOS      🔼   Gazprom      Noel       Kim  Schurrle   
11 2023-04-06    DARKS      🔽     Gilet      Mate    Reaper      None   
12 2023-04-13  BLANCOS      ⏫       Kim      Calz  

In [113]:
/* 
df = teams
pair_counts = {}

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    players = row[['Player1', 'Player2', 'Player3', 'Player4', 'Player5']].tolist()

    # Generate unique pairs of players
    player_pairs = [(p1, p2) for p1 in players for p2 in players if p1 != p2]

    for pair in player_pairs:
        if pair in pair_counts:
            pair_counts[pair] += 1
        else:
            pair_counts[pair] = 1

# Create a new DataFrame from the pair counts
pair_counts_df = pd.DataFrame({'Player Pair': list(pair_counts.keys()), 'Count': list(pair_counts.values())})

# Display the resulting DataFrame
#pair_counts_df.sort_values("Count")

,Player Pair,Count
142,"(Peggy, Zola)",1
201,"(Keysey, Peggy)",1
152,"(Virlo, Peggy)",1
199,"(Peggy, Keysey)",1
145,"(Zola, Peggy)",1
144,"(Peggy, Virlo)",1
204,"(Gazprom, Peggy)",1
205,"(Peggy, Gazprom)",1
206,"(Peggy, Schurrle)",1
207,"(Schurrle, Peggy)",1


In [135]:
pair_counts = {}
df = teams
# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    result = row['Result']
    players = row[['Player1', 'Player2', 'Player3', 'Player4', 'Player5']].tolist()

    # Generate unique pairs of players
    player_pairs = [(p1, p2) for p1 in players for p2 in players if p1 != p2]

    for pair in player_pairs:
        if pair in pair_counts:
            pair_counts[pair][result] = pair_counts[pair].get(result, 0) + 1
        else:
            pair_counts[pair] = {result: 1}

# Create a new DataFrame from the pair counts
pair_counts_df = pd.DataFrame(pair_counts).fillna(0)

# Display the resulting DataFrame
pair_counts_df
transposed_df = pair_counts_df.T
transposed_df_cleaned = transposed_df.dropna(axis=0).sort_index(axis=0)
transposed_df_cleaned.to_csv("../outputs/groupings_raw.csv")
transposed_df_cleaned


🔼   ⏬   🔽   ⏫   ➖
Calz     Gazprom   3.0 0.0 1.0 2.0 0.0
         Gilet     1.0 0.0 2.0 0.0 0.0
         Golfer    1.0 0.0 0.0 0.0 0.0
         Helmet    4.0 0.0 4.0 1.0 0.0
         Keysey    1.0 0.0 0.0 1.0 0.0
         Kim       3.0 1.0 3.0 4.0 0.0
         Mate      2.0 0.0 2.0 5.0 0.0
         Noel      1.0 1.0 3.0 1.0 1.0
         Peggy     0.0 0.0 1.0 1.0 0.0
         Reaper    4.0 1.0 1.0 2.0 0.0
         Schurrle  1.0 0.0 0.0 1.0 0.0
         Snoop     0.0 0.0 2.0 2.0 0.0
         Virlo     2.0 1.0 2.0 2.0 1.0
         Zola      3.0 0.0 3.0 1.0 0.0
         NaN       6.0 0.0 4.0 1.0 2.0
Gazprom  Calz      3.0 0.0 1.0 2.0 0.0
         Gilet     3.0 2.0 1.0 1.0 0.0
         Golfer    1.0 0.0 1.0 0.0 0.0
         Helmet    1.0 2.0 0.0 0.0 0.0
         Keysey    4.0 0.0 4.0 0.0 0.0
         Kim       6.0 1.0 1.0 2.0 0.0
         Mate      2.0 0.0 3.0 1.0 0.0
         Noel      2.0 1.0 1.0 0.0 0.0
         Peggy     0.0 0.0 1.0 0.0 0.0
         Reaper    7.0 2.0 1.0 0.0 0.0
         Schurrle  5.0 1.0 2.0 1.0 0.0
         Snoop     2.0 1.0 3.0 1.0 0.0
         Virlo     2.0 0.0 0.0 3.0 0.0
         Zola      1.0 1.0 1.0 0.0 0.0
         NaN       9.0 1.0 4.0 1.0 0.0
Gilet    Calz      1.0 0.0 2.0 0.0 0.0
         Gazprom   3.0 2.0 1.0 1.0 0.0
         Golfer    1.0 0.0 2.0 1.0 0.0
         Helmet    0.0 3.0 2.0 1.0 0.0
         Keysey    1.0 0.0 2.0 1.0 0.0
         Kim       0.0 1.0 1.0 0.0 0.0
         Mate      0.0 1.0 5.0 0.0 0.0
         Noel      0.0 2.0 1.0 1.0 0.0
         Reaper    2.0 2.0 2.0 1.0 0.0
         Schurrle  1.0 0.0 0.0 2.0 0.0
         Snoop     2.0 0.0 3.0 0.0 0.0
         Virlo     1.0 0.0 2.0 1.0 0.0
         Zola      1.0 1.0 1.0 1.0 0.0
         NaN       3.0 0.0 4.0 2.0 0.0
Golfer   Calz      1.0 0.0 0.0 0.0 0.0
         Gazprom   1.0 0.0 1.0 0.0 0.0
         Gilet     1.0 0.0 2.0 1.0 0.0
         Helmet    0.0 1.0 1.0 0.0 0.0
         Keysey    0.0 1.0 1.0 1.0 0.0
         Kim       0.0 0.0 1.0 0.0 0.0
         Mate      0.0 1.0 4.0 0.0 0.0
         Noel      0.0 1.0 0.0 0.0 0.0
         Reaper    1.0 0.0 3.0 1.0 0.0
         Schurrle  0.0 1.0 2.0 1.0 0.0
         Snoop     1.0 1.0 3.0 0.0 0.0
         Virlo     1.0 1.0 1.0 0.0 0.0
         Zola      1.0 1.0 1.0 0.0 0.0
         NaN       1.0 0.0 4.0 0.0 0.0
Helmet   Calz      4.0 0.0 4.0 1.0 0.0
         Gazprom   1.0 2.0 0.0 0.0 0.0
         Gilet     0.0 3.0 2.0 1.0 0.0
         Golfer    0.0 1.0 1.0 0.0 0.0
         Keysey    2.0 0.0 0.0 0.0 0.0
         Kim       3.0 2.0 6.0 0.0 0.0
         Mate      3.0 2.0 6.0 1.0 0.0
         Noel      1.0 3.0 4.0 2.0 0.0
         Peggy     1.0 0.0 2.0 0.0 0.0
         Reaper    3.0 3.0 0.0 0.0 0.0
         Schurrle  4.0 1.0 3.0 1.0 0.0
         Snoop     2.0 1.0 4.0 1.0 0.0
         Virlo     3.0 1.0 0.0 0.0 0.0
         Zola      4.0 4.0 2.0 3.0 0.0
         NaN       5.0 1.0 6.0 2.0 0.0
Keysey   Calz      1.0 0.0 0.0 1.0 0.0
         Gazprom   4.0 0.0 4.0 0.0 0.0
         Gilet     1.0 0.0 2.0 1.0 0.0
         Golfer    0.0 1.0 1.0 1.0 0.0
         Helmet    2.0 0.0 0.0 0.0 0.0
         Kim       4.0 0.0 2.0 0.0 1.0
         Mate      1.0 1.0 3.0 1.0 1.0
         Noel      1.0 0.0 2.0 0.0 0.0
         Peggy     0.0 0.0 0.0 1.0 0.0
         Reaper    5.0 0.0 1.0 2.0 0.0
         Schurrle  2.0 1.0 1.0 1.0 0.0
         Snoop     0.0 1.0 2.0 0.0 0.0
         Zola      0.0 0.0 1.0 0.0 1.0
         NaN       7.0 0.0 1.0 0.0 1.0
Kim      Calz      3.0 1.0 3.0 4.0 0.0
         Gazprom   6.0 1.0 1.0 2.0 0.0
         Gilet     0.0 1.0 1.0 0.0 0.0
         Golfer    0.0 0.0 1.0 0.0 0.0
         Helmet    3.0 2.0 6.0 0.0 0.0
         Keysey    4.0 0.0 2.0 0.0 1.0
         Mate      2.0 2.0 8.0 3.0 1.0
         Noel      3.0 2.0 6.0 0.0 0.0
         Peggy     1.0 0.0 2.0 0.0 0.0
         Reaper    6.0 2.0 3.0 1.0 0.0
         Schurrle  5.0 0.0 3.0 1.0 0.0
         Snoop     0.0 1.0 3.0 2.0 0.0
         Virlo     0.0 1.0 1.0 2.0 0.0
         Zola      2.0 2.0 3.0 0.0 1.0
         NaN       9.0 1.0 5.0 1.0 1.0
Mate

In [115]:
def count_same_team(df, name_col, date_col, team_col,result_col):
    count = 0
    for idx, row in df.iterrows():
        name = row[name_col]
        date = row[date_col]
        team = row[team_col]
        result = row[result_col]
        full_list = []
        same_team_df = df[(df[team_col] == team) & (df[date_col] == date)]
        same_team_names = same_team_df[name_col].unique()
        if len(same_team_names) > 1 and name in same_team_names:
            ll = str(same_team_names)
            full_list.append(ll)
    return full_list
a = count_same_team(raw_data, 'Name', 'Date', 'Team','Result')
a

["['Schurrle' 'Zola' 'Reaper' 'Helmet']"]

In [116]:
pd.DataFrame(a)

,0
0,['Schurrle' 'Zola' 'Reaper' 'Helmet']


In [117]:
a

["['Schurrle' 'Zola' 'Reaper' 'Helmet']"]